In [1]:
import keras

Using TensorFlow backend.


In [2]:
#from keras.preprocessing import image
#from keras.engine import Layer
from keras.layers import Conv2D, Conv3D, UpSampling2D, InputLayer, Conv2DTranspose, Input, Reshape, merge, concatenate
from keras import optimizers
#from keras.layers import Activation, Dense, Dropout, Flatten
#from keras.layers.normalization import BatchNormalization
#from keras.callbacks import TensorBoard
from keras.models import Sequential, Model
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

#from time import time
import numpy as np
#import os
#import random
import tensorflow as tf
from PIL import Image, ImageFile
import matplotlib.pyplot as plt


In [3]:
train_dir = './dataset/train'
seed = 6

bw = ImageDataGenerator(rescale=1./255)
c = ImageDataGenerator(rescale=1./255)


x = bw.flow_from_directory(train_dir,
                          target_size=(224,224),
                          color_mode='grayscale',
                          class_mode=None,
                          seed=seed)

y = c.flow_from_directory(train_dir,
                          target_size=(224,224),
                          class_mode=None,
                          seed=seed)

train_generator = zip(x, y)



Found 681 images belonging to 1 classes.
Found 681 images belonging to 1 classes.


In [4]:
validation_dir = './dataset/validation'
seed = 6

bw = ImageDataGenerator(rescale=1./255)
c = ImageDataGenerator(rescale=1./255)


x = bw.flow_from_directory(validation_dir,
                          target_size=(224,224),
                          color_mode='grayscale',
                          class_mode=None,
                          seed=seed)

y = c.flow_from_directory(validation_dir,
                          target_size=(224,224),
                          class_mode=None,
                          seed=seed)

validation_generator = zip(x, y)

Found 211 images belonging to 1 classes.
Found 211 images belonging to 1 classes.


In [5]:
#Encoder
encoder_input = Input(shape=(224, 224, 1,))
encoder_output = Conv2D(64, (3,3), activation='relu', padding='same', strides=2)(encoder_input)
encoder_output = Conv2D(128, (3,3), activation='relu', padding='same')(encoder_output)
encoder_output = Conv2D(128, (3,3), activation='relu', padding='same', strides=2)(encoder_output)
encoder_output = Conv2D(256, (3,3), activation='relu', padding='same')(encoder_output)
encoder_output = Conv2D(256, (3,3), activation='relu', padding='same', strides=2)(encoder_output)
encoder_output = Conv2D(512, (3,3), activation='relu', padding='same')(encoder_output)
encoder_output = Conv2D(512, (3,3), activation='relu', padding='same')(encoder_output)
encoder_output = Conv2D(256, (3,3), activation='relu', padding='same')(encoder_output)

In [6]:
#Decoder
decoder_output = Conv2D(128, (3,3), activation='relu', padding='same')(encoder_output)
decoder_output = UpSampling2D((2, 2))(decoder_output)
decoder_output = Conv2D(64, (3,3), activation='relu', padding='same')(decoder_output)
decoder_output = UpSampling2D((2, 2))(decoder_output)
decoder_output = Conv2D(32, (3,3), activation='relu', padding='same')(decoder_output)
decoder_output = Conv2D(16, (3,3), activation='relu', padding='same')(decoder_output)
decoder_output = Conv2D(3, (3, 3), activation='tanh', padding='same')(decoder_output)
decoder_output = UpSampling2D((2, 2))(decoder_output)
model = Model(inputs=encoder_input, outputs=decoder_output)

In [7]:
#model.summary()

In [8]:
opt = tf.keras.optimizers.Adam(learning_rate=0.002)

model.compile(opt, loss='mse', metrics=['accuracy'])

In [ ]:
# steps_per_epoch: Total number of steps (batches of samples) to yield from generator
# before declaring one epoch finished and starting the next epoch.
# It should typically be equal to the number of samples if your dataset divided by the batch size.
steps = 30 ### 2)100/62%acc #### 1)21/50%acc
val_steps = 6
iterations = 500 ### 2)30 ### 1)20

history = model.fit_generator(train_generator,
                              steps_per_epoch=steps,
                              epochs=iterations,
                              validation_data=validation_generator,
                              validation_steps=val_steps)

Epoch 1/500
30/30 [==============================] - 72s 2s/step - loss: 0.1391 - accuracy: 0.4168 - val_loss: 0.0418 - val_accuracy: 0.2149
Epoch 2/500
30/30 [==============================] - 68s 2s/step - loss: 0.0320 - accuracy: 0.4109 - val_loss: 0.0228 - val_accuracy: 0.3306
Epoch 3/500
30/30 [==============================] - 66s 2s/step - loss: 0.0208 - accuracy: 0.4301 - val_loss: 0.0201 - val_accuracy: 0.4503
Epoch 4/500
30/30 [==============================] - 71s 2s/step - loss: 0.0183 - accuracy: 0.4400 - val_loss: 0.0177 - val_accuracy: 0.4263
Epoch 5/500
30/30 [==============================] - 67s 2s/step - loss: 0.0166 - accuracy: 0.4394 - val_loss: 0.0193 - val_accuracy: 0.4345
Epoch 6/500
30/30 [==============================] - 68s 2s/step - loss: 0.0161 - accuracy: 0.4609 - val_loss: 0.0161 - val_accuracy: 0.4539
Epoch 7/500
30/30 [==============================] - 69s 2s/step - loss: 0.0154 - accuracy: 0.4808 - val_loss: 0.0158 - val_accuracy: 0.4739
Epoch 8/500
3

Epoch 59/500
30/30 [==============================] - 69s 2s/step - loss: 0.0097 - accuracy: 0.5893 - val_loss: 0.0125 - val_accuracy: 0.5777
Epoch 60/500
30/30 [==============================] - 73s 2s/step - loss: 0.0100 - accuracy: 0.5964 - val_loss: 0.0123 - val_accuracy: 0.5573
Epoch 61/500
30/30 [==============================] - 68s 2s/step - loss: 0.0104 - accuracy: 0.5856 - val_loss: 0.0140 - val_accuracy: 0.5604
Epoch 62/500
30/30 [==============================] - 73s 2s/step - loss: 0.0101 - accuracy: 0.5932 - val_loss: 0.0124 - val_accuracy: 0.5644
Epoch 63/500
30/30 [==============================] - 70s 2s/step - loss: 0.0098 - accuracy: 0.5907 - val_loss: 0.0100 - val_accuracy: 0.5714
Epoch 64/500
30/30 [==============================] - 68s 2s/step - loss: 0.0095 - accuracy: 0.6055 - val_loss: 0.0123 - val_accuracy: 0.5523
Epoch 65/500
30/30 [==============================] - 74s 2s/step - loss: 0.0097 - accuracy: 0.5970 - val_loss: 0.0122 - val_accuracy: 0.5464
Epoch 

In [ ]:
model.save('testModel(30steps500epochs).h5')

In [ ]:
import matplotlib.pyplot as plt

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'bo', label='Traning acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training & validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'bo', label='Traning loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training & validation loss')
plt.legend()

plt.show()

In [ ]:
history

In [ ]:
test_dir = './dataset/test'
seed = 6

bw = ImageDataGenerator(rescale=1./255)
c = ImageDataGenerator(rescale=1./255)


test_x = bw.flow_from_directory(test_dir,
                          target_size=(224,224),
                          color_mode='grayscale',
                          class_mode=None,
                          seed=seed)


In [ ]:
testpic = model.predict_on_batch(test_x[0])

In [ ]:
plt.imshow(test_x[0][1],aspect="auto", cmap=plt.get_cmap('gray'))
plt.show()
#print(x[0])

In [ ]:
plt.imshow(testpic[1],aspect="auto")
plt.show()